In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [8]:
print("Trainning data size:", mnist.train.num_examples)
print("Validating data size:", mnist.validation.num_examples)
print("Testing data size:", mnist.test.num_examples)

#print("Example trainning data:", mnist.train.images[0])
print("Example traingning data label:", mnist.train.labels[0])

print("Trainning data shape:", mnist.train.images.shape)

Trainning data size: 55000
Validating data size: 5000
Testing data size: 10000
Example traingning data label: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Trainning data shape: (55000, 784)


In [9]:
batch_size = 100
xs, ys = mnist.train.next_batch(batch_size)

print("X shape:", xs.shape)
print("Y shape:", ys.shape)

X shape: (100, 784)
Y shape: (100, 10)
